## Perturbative footprint study: data creation

### Imports

In [1]:
# Standard imports
import numpy as np
import shutil
from ruamel.yaml import YAML
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from difflib import Differ
from matplotlib.colors import ListedColormap
import matplotlib.ticker as ticker
from matplotlib.backends.backend_pdf import PdfPages
import qrcode
import io
# Cern modules
import fillingpatterns as fp
import xtrack as xt

# Improve style
import matplotlib
import seaborn as sns

sns.set_theme(style="whitegrid")
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
#sns.set(font='Adobe Devanagari')
sns.set_context("paper", font_scale=0.8, rc={"lines.linewidth": 0.2, "grid.linewidth": 0.1})


matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
# Not italized latex
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["font.weight"] = "light"

%config InlineBackend.figure_format='svg'


### Load dataset

In [2]:
# Load dataframe for DA
path = "/afs/cern.ch/work/c/cdroin/private/DA_study_for_bologna_cluster/opt_flathv_75_1500_withBB_1p4_survival_convergence"
df_survival = pd.read_parquet(f"{path}/da.parquet")

# Round all numbers to 3 decimals
df_survival = df_survival.round(3)

# Get list of bunches and list of DA
s_bunch_nb = df_survival["bunch_nb"].unique()

In [3]:
df_survival

,normalized amplitude in xy-plane,q1,q2,nb,on_x1,oct_current,bunch_nb,chromaticity_x,chromaticity_y
name 1,,,,,,,,,
madx_000,4.344,62.316,60.321,1.400000e+11,250.0,60.0,252.0,15.0,15.0


### Get data for survival analysis

In [4]:
# Build dataframe of initial distribution
path = "/afs/cern.ch/work/c/cdroin/private/DA_study_for_bologna_cluster/opt_flathv_75_1500_withBB_1p4_survival_convergence/particles"
df_initial_distribution = pd.DataFrame()
for name in os.listdir(path):
    if name.endswith(".parquet"):
        df_initial_distribution = pd.concat(
            [df_initial_distribution, pd.read_parquet(f"{path}/{name}")], ignore_index=False
        )
df_initial_distribution

,particle_id,normalized amplitude in xy-plane,angle in xy-plane [deg]
0,0.0,4.0,1.5
1,1.0,4.0,3.0
2,2.0,4.0,4.5
3,3.0,4.0,6.0
4,4.0,4.0,7.5
...,...,...,...
13,9494.0,9.0,82.5
14,9495.0,9.0,84.0
15,9496.0,9.0,85.5
16,9497.0,9.0,87.0


In [5]:
# List all bunch folders and build a dictionnary containing the corresponding survival rate for each particle in each bunch
path = "/afs/cern.ch/work/c/cdroin/private/DA_study_for_bologna_cluster/opt_flathv_75_1500_withBB_1p4_survival_convergence"
d_bunch_survival = {}
for name, bunch_nb, chroma in zip(df_survival.index, df_survival["bunch_nb"], df_survival["chromaticity_x"]):
    if name in os.listdir(path):
        path_part = path + "/" + name
        # Find all folders (particles) starting with xsuite and merge the corresponding dataframes
        # df_input_particle = pd.DataFrame()
        df_output_particle = pd.DataFrame()
        for folder in os.listdir(path_part):
            if folder.startswith("xsuite") and not folder.endswith("lines"):
                df_output_particle = pd.concat(
                    [
                        df_output_particle,
                        pd.read_parquet(f"{path_part}/{folder}/output_particles.parquet"),
                    ],
                    ignore_index=False,
                )
        # Get the corresponding number of turns for each particle and DA retained
        df_output_particle = df_output_particle[["particle_id", "at_turn"]]

        # Merge with initial distribution
        df_output_particle = df_output_particle.merge(
            df_initial_distribution, on="particle_id", how="left"
        )
        d_bunch_survival[ (int(bunch_nb), int(chroma))] = df_output_particle
    

### Check survival plot for a few selected bunches

In [16]:
def plot_survival(dic_bunch_survival, bunch_nb, chroma, subtitle=""):
    # Build coordinates in x and y
    amplitude = dic_bunch_survival["normalized amplitude in xy-plane"].to_numpy()
    phase = dic_bunch_survival["angle in xy-plane [deg]"].to_numpy()
    x = amplitude * np.cos(phase * np.pi / 180)
    y = amplitude * np.sin(phase * np.pi / 180)
    survival = dic_bunch_survival["at_turn"].to_numpy()

    # Map survival to color
    cmap = sns.color_palette("Spectral", as_cmap=True)

    # Add transparency to colormap
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:, -1] = 0.9
    cmap = ListedColormap(my_cmap)

    # Finish mapping
    norm = matplotlib.colors.Normalize(vmin=0, vmax=5e6)
    colors = cmap(norm(survival))

    # Survival plot
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_title(subtitle)
    ax.set_xlabel(r"$x [\sigma]$")
    ax.set_ylabel(r"$y [\sigma]$")
    # Plot points
    ax.scatter(x, y, c=colors, s=1, alpha=0.5)

    # Plot DA, that is, the amplitude of the particle with the survival different from 1000000
    amplitude = amplitude[survival != 5000000]
    survival = survival[survival != 5000000]
    DA = np.min(amplitude)
    # Plot a circle with radius DA
    circle = plt.Circle((0, 0), DA, color="grey", fill=False, linewidth=0.5, linestyle="--")
    ax.add_artist(circle)
    plt.text(s="DA", fontsize=5, x=0.1, y=DA + 0.1)
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)

    # For nicer formatting of colorbar ticks
    def fmt(x, pos):
        a, b = "{:.0e}".format(x).split("e")
        b = int(b)
        return r"${} \times 10^{{{}}}$".format(a, b)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cb = fig.colorbar(
        sm,
        ax=ax,
        fraction=0.036,
        pad=0.04,
        format=ticker.FuncFormatter(fmt),
        ticks=np.linspace(0, 5000000, 6),
    )
    cb.ax.set_ylabel(
        "Number of turns before losing particle", rotation=270, labelpad=15, fontsize=7
    )
    cb.outline.set_visible(False)
    cb.ax.tick_params(labelsize=5, length=2, width=0.4, which="major")
    ax.set_aspect("equal")
    ax.grid()
    plt.suptitle(
        "Survival plot for bunch " + str(bunch_nb) + " with chromaticity " + str(chroma) + ", DA = " + str(DA),
        y=0.96,
    )


    plt.tight_layout()
    plt.show()



In [17]:
bunch =252
chroma = 15
DA = df_survival[(df_survival.bunch_nb == bunch_nb) & (df_survival.chromaticity_x == chroma)]["normalized amplitude in xy-plane"].values[0]
type_bunch_annot = ""
plot_survival(d_bunch_survival[(bunch_nb, chroma)], bunch_nb, chroma, subtitle=type_bunch_annot)